In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.offline as py
import requests
import os

I want to look at how PHRMA spending leading up to the 2018 midterms is correlated with potential interests, including opioid overdose, marijuana use and regulations. We'll start by looking for a potential correlation with opioid death rates and absolute number of deaths.

Overdose rates and totals from the CDC data portal. Citation as follows:

Centers for Disease Control and Prevention, National Center for Health Statistics. Multiple Cause of Death 1999-2016 on CDC WONDER Online Database, released December, 2017. Data are from the Multiple Cause of Death Files, 1999-2016, as compiled from data provided by the 57 vital statistics jurisdictions through the Vital Statistics Cooperative Program. Accessed at http://wonder.cdc.gov/mcd-icd10.html on Jan 4, 2018 4:27:00 PM

In [3]:
#Loading in overdose rate, total overdose deaths
overdose_by_county = pd.read_csv('overdose_by_county.csv', delimiter='\t')[:1000]
overdose_by_state = pd.read_csv('overdose_by_state.csv', delimiter='\t')[:51]

In [4]:
#State-level overdose statistics
overdose_by_state.describe()

,State Code,Deaths,Population,Crude Rate
count,51.000000,51.000000,5.100000e+01,51.000000
mean,28.960784,1318.921569,6.335834e+06,21.927451
std,15.832828,1336.585453,7.243849e+06,9.186361
min,1.000000,75.000000,5.855010e+05,7.700000
25%,16.500000,335.500000,1.757121e+06,14.950000
50%,29.000000,912.000000,4.436974e+06,20.100000
75%,41.500000,1657.000000,7.109536e+06,26.700000
max,56.000000,5094.000000,3.925002e+07,49.800000


In [5]:
#County-level overdose statistics
overdose_by_county.describe()

,State Code,County Code,Deaths,Population
count,1000.000000,1000.00000,1000.000000,1.000000e+03
mean,30.503000,30590.80800,61.550000,2.792844e+05
std,15.174263,15188.83238,101.227677,5.446659e+05
min,1.000000,1003.00000,10.000000,1.464000e+04
25%,18.000000,18062.50000,15.000000,6.514550e+04
50%,34.000000,34018.00000,25.000000,1.246855e+05
75%,42.000000,42061.50000,60.000000,2.710415e+05
max,56.000000,56037.00000,1130.000000,1.013792e+07


In [6]:
#Sort states by rate of overdose
overdose_by_state = overdose_by_state.sort_values('Crude Rate', axis=0, ascending=False)
overdose_by_state

,Notes,State,State Code,Deaths,Population,Crude Rate
48,NaN,West Virginia,54.0,912.0,1831102.0,49.8
8,NaN,District of Columbia,11.0,276.0,681170.0,40.5
35,NaN,Ohio,39.0,4477.0,11614373.0,38.5
38,NaN,Pennsylvania,42.0,4762.0,12784227.0,37.2
29,NaN,New Hampshire,33.0,495.0,1334795.0,37.1
21,NaN,Massachusetts,25.0,2379.0,6811779.0,34.9
20,NaN,Maryland,24.0,2089.0,6016447.0,34.7
17,NaN,Kentucky,21.0,1525.0,4436974.0,34.4
39,NaN,Rhode Island,44.0,330.0,1056426.0,31.2
7,NaN,Delaware,10.0,288.0,952065.0,30.3


In [7]:
overdose_by_county

,Notes,State,State Code,County,County Code,Deaths,Population,Crude Rate
0,NaN,Alabama,1.0,"Baldwin County, AL",1003.0,34.0,208563.0,16.3
1,NaN,Alabama,1.0,"Blount County, AL",1009.0,15.0,57704.0,Unreliable
2,NaN,Alabama,1.0,"Calhoun County, AL",1015.0,16.0,114611.0,Unreliable
3,NaN,Alabama,1.0,"Cullman County, AL",1043.0,20.0,82471.0,24.3
4,NaN,Alabama,1.0,"DeKalb County, AL",1049.0,19.0,70900.0,Unreliable
5,NaN,Alabama,1.0,"Escambia County, AL",1053.0,11.0,37728.0,Unreliable
6,NaN,Alabama,1.0,"Etowah County, AL",1055.0,18.0,102564.0,Unreliable
7,NaN,Alabama,1.0,"Jefferson County, AL",1073.0,256.0,659521.0,38.8
8,NaN,Alabama,1.0,"Lauderdale County, AL",1077.0,13.0,92318.0,Unreliable
9,NaN,Alabama,1.0,"Lee County, AL",1081.0,11.0,158991.0,Unreliable


OK, so let's first see if we can establish that big pharma is targeting Congressional races by the year. If we can do that, it may be possible to demonstrate that they're also shifting their money year after year depending on *where* the deaths are occurring.

In [21]:
#Set up Center for Responsive Politics (OpenSecrets) API
from crpapi import CRP
OPENSECRETS_API_KEY = os.environ['OPENSECRETS_API_KEY']
PROPUBLICA_CAMPAIGN_FINANCE_API_KEY = os.environ['PROPUBLICA_CAMPAIGN_FINANCE_API_KEY']
PHARMA_MANUFACTURING = 'H4300'
PHARMA_WHOLESALE = 'H4400'
opensecrets = CRP(OPENSECRETS_API_KEY)

In [22]:
#Set up data scraping parameters
years = [2013,2014,2015,2016,2017,2018]
candidates_uri = 'https://api.propublica.org/campaign-finance/v1/{cycle}/candidates/search'
print(candidates_uri.format(cycle=2012))
payload = {
    'X-API-Key':PROPUBLICA_CAMPAIGN_FINANCE_API_KEY
}
r = requests.get(candidates_uri.format(cycle=2012), params=payload)
r.json()

https://api.propublica.org/campaign-finance/v1/2012/candidates/search


{'message': 'Unauthorized'}